In [1]:
import os
import cv2
import numpy as np

In [2]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

2023-05-01 09:22:27.534699: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 09:22:27.647594: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 09:22:27.648393: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 09:22:28.347441: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Replicate results

In [3]:
np.random.seed(42)

ROOT_DIR = '../data/images'
TRAIN_DIR = os.path.join(ROOT_DIR, 'train')
TEST_DIR = os.path.join(ROOT_DIR, 'test')
input_shape = (32, 32, 1)
batch_size = 32
epochs = 30

# Build and compile model

In [4]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=input_shape, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(3, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


2023-05-01 09:25:45.955057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-01 09:25:45.955718: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Load data

In [5]:
def load_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # Fit to input size
    img = cv2.resize(img, (32, 32))
    img = np.expand_dims(img, axis=-1)
    return img.astype(np.float32)

In [6]:
def one_hot_encode(labels):
    mapper = {'blank': 0, 'nough': 1, 'circle': 2}
    encoded = [mapper[label] for label in labels]
    target = to_categorical(encoded)
    return target

In [7]:
def load_data(root_dir, shuffle=True):
    X, y = [], []
    for root, dirs, files in os.walk(root_dir):
        for class_dir in dirs:
            image_dir = os.path.join(root_dir, class_dir)
            y.extend(np.tile(class_dir, len(os.listdir(image_dir))))
            for img_fn in os.listdir(image_dir):
                img = load_img(os.path.join(image_dir, img_fn))
                X.append(img)
    y = one_hot_encode(y)
    if shuffle:
        # Combine to maintain order
        data = list(zip(X, y))
        np.random.shuffle(data)
        X, y = zip(*data)
    return np.asarray(X), np.asarray(y)


In [8]:
print('Loading data...')
X_train, y_train = load_data(TRAIN_DIR)
X_test, y_test = load_data(TEST_DIR)
print('{} instances for training'.format(len(X_train)))
print('{} instances for evaluation'.format(len(X_test)))

Loading data...


ValueError: zero-size array to reduction operation maximum which has no identity

# Create more instances since our dataset is VERY limited

In [9]:
train_val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1 / 255,
    validation_split=0.2)

train_generator = train_val_datagen.flow(
    X_train, y_train, batch_size=batch_size, subset='training')

val_generator = train_val_datagen.flow(
    X_train, y_train, batch_size=batch_size, subset='validation')

NameError: name 'X_train' is not defined

# Train and evaluate

In [10]:
callbacks = [
    EarlyStopping(patience=5, verbose=1, restore_best_weights=True)]

print('Training model...')
history = model.fit_generator(
    train_generator,
    steps_per_epoch=128,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=32,
    callbacks=callbacks)

print('Evaluating model...')

Training model...


NameError: name 'train_generator' is not defined

# Load data

In [11]:
test_datagen = ImageDataGenerator(rescale=1 / 255)
X_test, y_test = next(test_datagen.flow(X_test, y_test, batch_size=len(X_test)))

loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Crossentropy loss: {:0.3f}'.format(loss))
print('Accuracy: {:0.3f}'.format(acc))

NameError: name 'X_test' is not defined

# Save model

In [ ]:
model.save('../data/model.h5')
print('Saved model to disk')
